In [12]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation,PillowWriter

from IPython.display import HTML
import numba

@numba.njit
def next_cell_value(arr,r,c):
    height, width = arr.shape
    cell_sum = -arr[r,c]
    for dh in range (-1, 2):
        for dw in range(-1, 2):
            cell_sum += arr[(r + dh) % height, (c + dw) % width]

    if arr[r,c] == 1 and (cell_sum == 2 or cell_sum == 3):
        return 1
    if arr[r,c] == 0 and (cell_sum == 3):
        return 1
    return 0

@numba.njit(parallel = True)
def next_step(in_arr, out_arr):
    height, width = in_arr.shape
    for h in numba.prange(height):
        for w in numba.prange(width):
            out_arr[h,w] = next_cell_value(in_arr,h,w)
    return out_arr


In [13]:
ic = np.random.randint(0, 2, (50, 50))
buffer = np.zeros_like(ic)

fig = plt.figure(figsize = (10, 10))
data = plt.imshow(ic)
plt.close()

def frame(i):
    global ic, buffer
    next_step(ic, buffer)
    ic, buffer = buffer, ic
    data.set_data(ic)
    return data,

animation = FuncAnimation(fig, frame, frames = 100, blit = True)
HTML(animation.to_jshtml())

In [14]:
@numba.njit(parallel = True)
def one_step(l):
    ic = np.random.randint(0, 2,(l,l))
    buffer = np.zeros_like(ic)

    next_step(ic,buffer)
    ic, buffer = buffer, ic 

In [15]:
one_step(5000)

@numba.jit(nopython = True)
@numba.njit
